### กลุ่ม Diabetes Prediction 
1. 6610402205 นายรักษิต รุ่งรัตนไชย หมู่ 1
2. 6610402132 นายบวรรัตน์ ตั้งนรารัชชกิจ หมู่ 1
3. 6610401985 นายไชยวัตน์ หนูวัฒนา หมู่ 1 

### วัตถุประสงค์ของระบบต้นแบบ
กลุ่มของพวกเราเล็งเห็นถึงความสำคัญทางด้านสุขภาพของประชาชนคนไทย ในปัจจุบันคนไทยประสบกับปัญหาสุขภาพมากมายแต่หนึ่งปัญหาที่สร้างความเสียหายให้กับสุขภาพและคนไทยเป็นกันมากที่สุดนั่นคือโรคเบาหวาน พฤติกรรมการบริโภคของคนไทยในสมัยนี้ก็เป็นปัจจัยหนึ่งที่ทำให้ปริมาณผู้ป่วยโรคเบาหวานเพิ่มมากยิ่งขึ้น พวกเราจึงอยากทำระบบต้นแบบที่ช่วยแบ่งเบาภาระของเจ้าหน้าที่ในหน่วยงานสาธารณะสุข หมอ พยาบาล ในการวินิจฉัยโรคเบาหวาน ระบบต้นแบบอาจจะไม่สามารถชี้ชัดได้ 100 เปอเซนแต่สามารถคัดกรองผู้ป่วยและแบ่งเบาภาระได้ส่วนหนึ่งซึ่งพวกเราคิดว่ามันจะเป็นประโยชน์อย่างมากในการคัดกรองผู้ป่วย

### ลิงค์ไปยังข้อมูลที่จะใช้ในระบบต้นแบบ
Link to data: https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset

In [ ]:
import pandas as pd

diabetes = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")
diabetes.head()

### ปั​ญห​า ML ของตั​วเอ​งเป็นปั​ญหา​ป​ระเภ​ทใด (classificati​on/regres​sion/clust​ering/ge​neration)
ปัญหา ML ของกลุ่มพวกเราเป็นปัญหาแบบ classification binary เนื่องจากเป็นการทำนายว่าข้อมูลของบุคคลที่นำเข้ามาในโมเดลนั้นเป็นหรือไม่เป็นโรคเบาหวานจึงเป็นการ classification binary อย่างชัดเจน

### f​eat​ur​es ที่ใช้ในการทำนา​ยมีอะ​ไรบ้าง
ก่อนที่เราจะสามารถดูค่าความสำคัญของแต่ละฟีเจอร์ได้นั้น เราต้องทำ Feature Engineer กับข้อมูลให้เรียบร้อยก่อนโดยมีขั้นตอนดังต่อไปนี้

แยกชุดข้อมูลออกเป็น ชุดข้อมูลเรียนรู้(train) ชุดข้อมูลทดสอบ(test) และชุดข้อมูลตรวจสอบความแม่นยำของโมเดล(validation)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, stratify=diabetes["Diabetes_binary"]
                               , test_size=0.3, random_state=1234)
test, validation = train_test_split(test, stratify=test["Diabetes_binary"],
                                    test_size=0.3, random_state=1234)
len(train), len(test), len(validation)

เนื่องจากข้อมูลของกลุ่มเรานั้น ขาดความสมดุลอย่างมากสังเกตได้จาก จำนวนของ Label ที่มีจำนวนไม่เท่ากัน

In [ ]:
diabetes.Diabetes_binary.value_counts()

ดังนั้นต้องทำให้ชุดข้อมูลที่นำมาใช้งานนั้นมีความสมดุลกันก่อนด้วยการทำ Under Sampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
resampler = RandomUnderSampler(random_state=1234)

feature = diabetes.drop(columns="Diabetes_binary").columns
label = "Diabetes_binary"

train[feature], train[label] = resampler.fit_resample(train[feature], train[label])
train.dropna(inplace=True)

train.head()

ขั้นตอนต่อมาคือการทำ Pipeline ของ Logistic Regression และ Decision Tree โดยเหตุผลที่เลือกทำ pipeline ทั้ง 2 โมเดลเพราะ ต้องการหาโมเดลที่มีความแม่นยำสูงและไม่ Overfit จนเกินไป

สร้าง Pipeline ของ Logistic Regression

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression

def create_pipline_logistic():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["PhysHlth"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["MentHlth"]),
            remainder="passthrough"
        ),
        LogisticRegression(max_iter=500)
    )

logistic_pipeline = create_pipline_logistic()
logistic_pipeline

สร้างโมเดล Logistic Regression และตรวจสอบความแม่นยำ

In [ ]:
logistic_pipeline.fit(train[feature], train[label])
logistic_pipeline.score(validation[feature], validation[label])

สร้าง Pipeline ของ SDG Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

def create_pipline_sgd():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["PhysHlth"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["MentHlth"]),
            remainder="passthrough"
        ),
        SGDClassifier()
    )

sgd_pipline = create_pipline_sgd()
sgd_pipline

สร้างโมเดล SDG Classifier และตรวจสอบความแม่นยำ

In [ ]:
sgd_pipline.fit(train[feature], train[label])
sgd_pipline.score(validation[feature], validation[label])

สร้าง Pipeline ของ Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

def create_pipline_decisiontree():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["PhysHlth"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["MentHlth"]),
            remainder="passthrough"
        ),
        DecisionTreeClassifier()
    )

decision_pipeline = create_pipline_decisiontree()
decision_pipeline

สร้างโมเดล Decision Tree และตรวจสอบความแม่นยำ

In [ ]:
decision_pipeline.fit(train[feature], train[label])
decision_pipeline.score(validation[feature], validation[label])

ตรวจสอบค่าความสำคัญของแต่ละฟีเจอร์จากโมเดล Logistic Regression เพราะเป็นโมเดลมีความแม่นยำสูงที่สุด

In [ ]:
from sklearn.inspection import permutation_importance

importance = permutation_importance(logistic_pipeline, test[feature], test[label], random_state=1234)
importance = pd.Series(importance.importances_mean, index=feature)
importance.sort_values(ascending=False)

โดยเลือก 7 ฟีเจอร์แรกที่มีค่าความสำคัญสูงสุดมาใช้งาน

In [ ]:
importance_features = ["GenHlth", "BMI", "HeartDiseaseorAttack", "DiffWalk", "HvyAlcoholConsump", "Stroke", "NoDocbcCost"]

### เฉ​ลย (label) ที่ต้​องกา​รทำนา​ยคื​ออ​ะไร
สิ่งที่ต้องการทำนายคือ 0 หรือ 1 (0 คือไม่เป็นโรคเบาหวานและ 1 คือเป็นโรคเบาหวาน)

In [ ]:
labels = ["ไม่เป็นโรคเบาหวาน", "เป็นโรคเบาหวาน"]

for i in diabetes.Diabetes_binary.unique().astype(int):
    print(f"{i} = {labels[i]}")

### ระบบของตัวเองสาม​ารถใช้ al​gorith​m อะไรได้บ้าง เพ​ราะเห​ตุใด

เกณฑ์ที่กลุ่มเราใช้ในการเลือกใช้โมเดลจะอิงจาก Cheat Sheet ของ Scikit-Learn ดังภาพต่อไปนี้:

<img src="https://scikit-learn.org/1.3/_static/ml_map.png" alt="sklearn-cheat-sheet" width="700">

เนื่องจากชุดข้อมูลที่กลุ่มเรานำมาใช้นั้นเป็นแบบ Classification หรือแบ่งแยกประเภท และมีเป็นข้อมูลที่มี Labeled จากรูปจะมี `SGD Classifier` และ `Kernel Approximation` ที่สามารถนำมาใช้กับข้อมูลของกลุ่มเราได้ และยังมี Algorithm อื่นๆอีกที่สามารถนำมาใช้งานกับข้อมูลของกลุ่มเราได้แต่ไม่ได้อยู่ใน Cheat Sheet นี้เช่น `Logistic Regression` และ `Decision Tree` เป็นต้น

### แต่ละ algor​ithm มี hype​rpar​am​eter อะไ​รบ้าง ปรับเป็​นอะไรไ​ด้บ้าง
แต่ละ algorithm มี hyperparameter ดังต่อไปนี้

In [ ]:
logistic_pipeline.get_params()

In [ ]:
sgd_pipline.get_params()

In [ ]:
decision_pipeline.get_params()

ทำการ Tunning ให้กับโมเดล Logistic Regresion เพื่อหา hyperparameter ที่ทำให้โมเดลมีประสิทธิภาพมากขึ้น

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings("ignore")
pipeline = create_pipline_logistic() 

param_dist = {
     'columntransformer__kbinsdiscretizer-1__strategy': ['uniform', 'quantile', 'kmeans'],
     'columntransformer__kbinsdiscretizer-2__strategy': ['uniform', 'quantile', 'kmeans'],
     'columntransformer__kbinsdiscretizer-3__strategy': ['uniform', 'quantile', 'kmeans'],
     'columntransformer__kbinsdiscretizer-1__encode': ['ordinal', 'onehot', 'onehot-dense'],
     'columntransformer__kbinsdiscretizer-2__encode': ['ordinal', 'onehot', 'onehot-dense'],
     'columntransformer__kbinsdiscretizer-3__encode': ['ordinal', 'onehot', 'onehot-dense'],
     'columntransformer__kbinsdiscretizer-1__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-2__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-3__n_bins': [5, 10, 20],
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logisticregression__penalty': ['none', 'l1', 'l2', 'elasticnet'],
    'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'logisticregression__max_iter': [100, 200, 300],
    'logisticregression__tol': [1e-4, 1e-3, 1e-2]
}

search = RandomizedSearchCV(pipeline, param_dist, n_iter=10, random_state=0)
search.fit(train[feature], train[label])
search.best_params_, f"{search.best_score_:.3f}"

ปรับแต่ง hyperparameter ตามผลที่ได้จากการ tunning model

ทำการ Tunning ให้กับโมเดล SGD Classifier เพื่อหา hyperparameter ที่ทำให้โมเดลมีประสิทธิภาพมากขึ้น

In [ ]:
warnings.filterwarnings("ignore")
pipeline = create_pipline_sgd()

param_dist = {
    'columntransformer__kbinsdiscretizer-1__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-2__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-3__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-1__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-2__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-3__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-1__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-2__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-3__n_bins': [5, 10, 20],
    'sgdclassifier__loss' : ['hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'sgdclassifier__penalty': ['l2', 'l1', 'elasticnet', None]
}

search = RandomizedSearchCV(pipeline, param_dist, n_iter=20, random_state=0)
search.fit(train[feature], train[label])
search.best_params_, f"{search.best_score_:.3f}"

In [ ]:
search.best_estimator_.score(validation[feature], validation[label])

ทำการ Tunning ให้กับโมเดล Decision Tree เพื่อหา hyperparameter ที่ทำให้โมเดลมีประสิทธิภาพมากขึ้น

In [ ]:
warnings.filterwarnings("ignore")
pipeline = create_pipline_decisiontree()

param_dist = {
    'columntransformer__kbinsdiscretizer-1__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-2__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-3__strategy': ['uniform', 'quantile', 'kmeans'],
    'columntransformer__kbinsdiscretizer-1__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-2__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-3__encode': ['ordinal', 'onehot', 'onehot-dense'],
    'columntransformer__kbinsdiscretizer-1__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-2__n_bins': [5, 10, 20],
    'columntransformer__kbinsdiscretizer-3__n_bins': [5, 10, 20],
    'decisiontreeclassifier__criterion':['gini', 'entropy', 'log_loss'],
}

search = RandomizedSearchCV(pipeline, param_dist, n_iter=20, random_state=34)
search.fit(train[feature], train[label])
search.best_params_, f"{search.best_score_:.3f}"

In [ ]:
search.best_estimator_.score(validation[feature], validation[label])

### คำตอบที่ได้จากการทำนายของ model เบื้องต้นปกติหรือไม่ อย่างไร ต้องมีการ p​os​t-p​roc​ess หรือไม่ อย่างไร
เริ่มด้วยการ
1. ทดสอบประสิทธิภาพของโมเดลเบื้องต้น 
2. นำโมเดลเบื้องต้นมาทำนายเพื่อดูความแม่นยำ

In [ ]:
test_efficient_logistic = create_pipline_logistic()
%time test_efficient_logistic.fit(train[feature], train[label])
test_efficient_logistic.score(validation[feature], validation[label])

In [ ]:
samples = validation.sample(5, random_state=1234)
samples

In [ ]:
predicted = logistic_pipeline.predict(samples)
predicted

ผลจากการทำนายเบื้องต้นพบว่ายังคงมีการทำนายผิดพลาดเกิดขึ้นอยู่จึงต้องทำการ post process เพื่อเพิ่มความแม่นยำของโมเดลโดยการ drop บางฟีเจอร์ที่มีความสำคัญน้อยออกไปใช้เฉพาะฟีเจอร์ที่มีความสำคัญมาก (ดูความสำคัญของฟีเจอร์จากการทำ feature importance ก่อนหน้านี้)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression

def create_new_pipline_logistic():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            remainder="passthrough"
        ),
        LogisticRegression(max_iter=500)
    )

new_logistic_pipeline = create_new_pipline_logistic()
new_logistic_pipeline

In [ ]:
new_logistic_pipeline.get_params()

In [ ]:
%time new_logistic_pipeline.fit(train[importance_features], train[label])
new_logistic_pipeline.score(validation[importance_features], validation[label])

In [ ]:
warnings.filterwarnings("ignore")
pipeline = create_new_pipline_logistic() 

param_dist = {
     'columntransformer__kbinsdiscretizer__strategy': ['uniform', 'quantile', 'kmeans'],
     'columntransformer__kbinsdiscretizer__encode': ['ordinal', 'onehot', 'onehot-dense'],
     'columntransformer__kbinsdiscretizer__n_bins': [5, 10, 20],  # Adjusted to correct key
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'logisticregression__penalty': ['none', 'l1', 'l2', 'elasticnet'],
    'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'logisticregression__max_iter': [100, 200, 300],
    'logisticregression__tol': [1e-4, 1e-3, 1e-2],
}

search = RandomizedSearchCV(pipeline, param_dist, n_iter=10, random_state=0)
search.fit(train[importance_features], train[label])
search.best_params_, f"{search.best_score_:.3f}"

In [ ]:
search.best_estimator_.score(validation[importance_features], validation[label])

หลังจากการทำ post process โดยการทำ Hyperparameters Tuning พบว่า score ความแม่นยำเพิ่มขึ้นเล็กน้อยและ มีระยะเวลาที่ใช้ในการสร้างโมเดลที่เร็วขึ้น

### ปร​ะเด็น​อื่น ๆ ที่เกี่ยว​ข้องกั​บข้อมูลของ​ตัวเอง

* ในขั้นตอนการหาค่าความสำคัญของฟีเจอร์จากการทดสอบหาค่าความสำคัญจากโมเดล SDG Classifier พบว่าผลลัพธ์ที่ได้มีค่าเป็น 0.0 ทั้งหมด ไม่ทราบว่าเพราะเหตุผลใดจึงได้ผลลัพธ์ออกมาเป็นแบบนั้นจึงเลือกที่จะใช้แค่ 2 โมเดลนั่นคือ Logistic regression และ Decision tree ซึ่งทั้งสองโมเดลนี้สามารถหา feature importance ได้

* อีกประเด็นที่ค้นพบก็คือในตอนแรกข้อมูลเกิดความ imbalance ซึ่งคิดว่าไม่ได้ส่งผลอะไรเพราะตอนแบ่ง train test ได้ทำการแบ่งอย่างถูกต้องต้องแล้วตามที่งานก่อนหน้านี้ที่ได้ทำไปซึ่งผลการทดสอบโมเดลก็อยู่ในคะแนนที่ดีประมาณ 0.8 กว่าๆ แต่เมื่อทำการทดลองใช้โมเดลกับข้อมูลที่มีแต่คนที่เป็นโรคเบาหวานปรากฏว่าโมเดลทำนายไม่แม่นเลย score ลดลงเหลือ 0.1 กว่าๆแปลว่าโมเดลพยายามเดา label ให้เป็น 0 เพราะในชุด train มีจำนวน label 0 มากกว่า 1 หลายเท่าตัว จึงทำการแก้ imbalace โดยใช้วิธี `under sampling` ให้จำนวนคนเป็นกับไม่เป็นโรคเบาหวานเท่าๆกัน (ซึ่งแลกมาด้วยการที่ข้อมูลหายไปจำนวนมากจาก 2 แสนกว่า เหลือ 5 หมื่นกว่า) แต่สิ่งที่ได้มาคือโมเดลทำนายได้ดีขึ้นวัด score อยู่ที่ 0.7 นิดๆทั้งชุดที่มีแต่คนเป็นโรคและชุดของคนที่ไม่เป็นโรค (อัลกอริธึมที่ใช้คือ Logistic Regression)

### ก​าร​มีส่วน​ร่วมของสม​าชิกแต่ละ​คนในกลุ่ม (แต่ล​ะคนทำอะไรบ้าง)
- นายรักษิต รุ่งรัตนไชย ทำหน้าที่เขียนในหัวข้อ 1 2 3 4 8 9 10
- นายบวรรัตน์ ตั้งนรารัชชกิจ ทำหน้าที่เขียนในหัวข้อ 5 6 7 ทำการ tunning model เพื่อหา hyperparameter ที่ดีที่สุด
- นายไชยวัตน์ หนูวัฒนา เขียน pipeline เพื่อใช้ในการสร้าง model, หา feature importance และ tunning hyperparameter

### กา​รเปิ​ดเผ​ยกา​รใช้เค​รื่อ​งมือปัญ​ญาป​ระดิษฐ์ (ใช้อะไร ใช้เพื่ออะไร ใช้อย่างไร, pro​mpt อย่างไร)
ในการทำงานครั้งนี้ ไม่ได้ใช้ปัญญาประดิษฐ์ในการทำงานแต่อย่างใด